In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import torch
import torch.nn as nn
import torch_geometric as tg
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GCNConv
from torch_geometric.utils import add_self_loops, degree
from torch.nn import init
from torch.utils.data import Dataset, DataLoader, Subset, ConcatDataset
import torch.optim as optim
from torch.autograd import Function

In [2]:
def save_data(data, file_path):
    with open(file_path , 'wb') as f:
        pickle.dump(data,f)
        f.close()

In [3]:
class TimeSeriesDataset(Dataset):
    def __init__(self, inputs, targets, dist, dist_arg, masks, min_vals, max_vals, mask_dist_max, class_labels):
        self.inputs = inputs
        self.targets = targets
        self.dist = dist
        self.dist_arg = dist_arg
        self.masks = masks
        self.min_vals = min_vals
        self.max_vals = max_vals
        self.mask_dist_max = mask_dist_max
        self.class_labels = class_labels
    
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        # 获取输入和对应的目标数据
        x = self.inputs[idx]
        y = self.targets[idx]
        d_ = self.dist[idx]
        d_arg = self.dist_arg[idx]
        mask = self.masks[idx]
        min_ = self.min_vals[idx]
        max_ = self.max_vals[idx]
        mask_d_ = self.mask_dist_max[idx]
        class_ = self.class_labels[idx]
        return x, y, d_, d_arg, mask, min_, max_, mask_d_, class_

In [4]:
def sort_pooling(x, k, num_nodes):
    # x SHAPE: (Batch_num * Max_node_num), (hidden_channels_2)
    #-------------------> Batch_num, Max_node_num, (hidden_channels_2)
    x = x.reshape(-1, num_nodes, x.shape[-1])
    norms = torch.norm(x, p=2, dim= -1)
    #------------------------------------------------------------
    _, sorted_indices = torch.sort(norms, dim= -1, descending=True)
    
    sorted_x = torch.gather(x, dim=1, index=sorted_indices.unsqueeze(-1).expand(-1, -1, x.size(-1)))
    top_k_x = sorted_x[:, :k , :]
    # top_k_x SHAPE: Batch_num, K, (hidden_channels_2)
    
    return top_k_x

In [5]:
class GradientReversalFunction(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        grad_input = grad_output.neg() * ctx.alpha  # 梯度取反
        return grad_input, None

class GradientReversalLayer(nn.Module):
    def __init__(self, alpha=None):
        super(GradientReversalLayer, self).__init__()
        self.alpha = alpha

    def forward(self, x):
        return GradientReversalFunction.apply(x, self.alpha)

In [6]:
#class GradientReversalLayer(Function):
    #def forward(ctx, x, alpha):
        #ctx.alpha = alpha
        #return x.view_as(x)
    
    
    #def backward(ctx, grad_output):
        #output = grad_output.neg() * ctx.alpha  # 反转梯度
        #return output, None

In [7]:
class Domain_Classifier(nn.Module):
    def __init__(self, hidden_channels_2, class_hidden_1, class_hidden_2, k, num_nodes, alpha, drop_out = True):
        super(Domain_Classifier, self).__init__()
        self.k = k
        self.drop_out = drop_out
        self.num_nodes = num_nodes
        self.fc_1 = nn.Sequential(nn.Linear(k * hidden_channels_2, class_hidden_1), nn.LeakyReLU())  
        #self.fc_2 = nn.Sequential(nn.Linear(class_hidden_1, class_hidden_2), nn.LeakyReLU()) 
        self.fc_3 = nn.Sequential(nn.Linear(class_hidden_1, 1), nn.LeakyReLU())  
        self.grl = GradientReversalLayer(alpha) 
        
        for name, module in self.named_modules():
            if isinstance(module, nn.Linear):
                module.weight.data = init.kaiming_uniform_(module.weight.data, nonlinearity='leaky_relu')
                if module.bias is not None:
                    module.bias.data = init.constant_(module.bias.data, 0.0)                    
        
        
    def forward(self, x):
        #x shape: (Batch_num * Max_node_num), 7, hidden_channels_2
        x = x 
        x = x[:, -1, :]
        #---------------------------------------------------------
        # top_k_x SHAPE: Batch_num, K, hidden_channels_2
        x = sort_pooling(x, self.k, self.num_nodes) 
        #------------------------------------------------------------
        x = x.flatten(start_dim=1, end_dim=2)
        #-----------------------------------------------------------
        #Shape: Batch_num, hidden_dimen
        #-----------------------------------------------------------
        x = self.grl(x)
        #-----------------------------------------------------------
        x = self.fc_1(x)
        if self.drop_out:
            x = F.dropout(x, training=self.training, p = 0.2)
        #------------------------------------------------------------
        #x = self.fc_2(x)
        #if self.drop_out:
            #x = F.dropout(x, training=self.training, p = 0.2)

        
        # SHAPE: Batch_num, 1
        x = self.fc_3(x) 
      
        return x

In [8]:
class NonLinearOperate(nn.Module):
    def __init__(self, input_dimen, hidden_dimen, output_dimen):
        super(NonLinearOperate, self).__init__()  #类NonLinearLayer继承父类nn.Module的初始化方法
        self.layer_1 = nn.Linear(input_dimen, hidden_dimen)
        self.layer_2 = nn.Linear(hidden_dimen, output_dimen)
        self.acti_func = nn.ReLU()
        for m in self.modules():#遍历所有子模块
        #Check if each sub-module is an example of the class nn.Linear
            if isinstance(m, nn.Linear):
                m.weight.data = init.kaiming_normal_(m.weight.data, nonlinearity='relu')
                if m.bias is not None:
                    m.bias.data = init.constant_(m.bias.data, 0.0)
            
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.acti_func(x)
        x = self.layer_2(x)
        x = self.acti_func(x)
        
        return x

In [9]:
class PGNN_Layer(nn.Module):
    def __init__(self, hidden_dimen, output_dimen, anchor_num, drop_out = True):
        super(PGNN_Layer, self).__init__()
        self.drop_out = drop_out
        self.output_dimen = output_dimen
        #self.distance_calculate = NonLinearOperate(1, output_dimen, 1)
        self.acti_func = nn.LeakyReLU()
        self.linear_hidden = nn.Linear(2 * anchor_num, hidden_dimen)
        self.out_layer = nn.Linear(hidden_dimen, output_dimen)
        
        self.linear_structure = nn.Linear(hidden_dimen, 1)

        
        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.weight.data = init.kaiming_uniform_(m.weight.data, nonlinearity='leaky_relu')
                if m.bias is not None:
                    m.bias.data = init.constant_(m.bias.data, 0.0)                
                    
    def forward(self, node_features, dists_max, dists_argmax, mask_dist_max):
        # node feature SHAPE: Batch-size, 28, Max_node_num
        # Dist_max SHAPE: Batch-size, 28, Max_node_num, 36
        # mask_dist_max SHAPE: Batch-size, 28, Max_node_num, 36
        batch_size, num_days, max_node_num = node_features.size()
        #dists_max = self.distance_calculate(dists_max.unsqueeze(-1)).squeeze()

        indices_expanded = dists_argmax.flatten(start_dim=2)
        batch_indices = torch.arange(batch_size).view(-1, 1, 1).expand(-1, num_days, indices_expanded.shape[-1])
        day_indices = torch.arange(num_days).view(1, -1, 1).expand(batch_size, -1, indices_expanded.shape[-1]) 
        
        subset_features = node_features[batch_indices, day_indices, indices_expanded]


        #---------------------------------------------------------------
        subset_features = subset_features.reshape(subset_features.shape[0], subset_features.shape[1], dists_argmax.shape[2], dists_argmax.shape[3])
        #message SHAPE: Batch-size, 28, Max_node_num, 36
        messages = subset_features * dists_max * mask_dist_max
        feature_self = node_features.unsqueeze(-1).repeat(1, 1, 1, dists_max.shape[-1])
        
        #messages SHAPE: Batch-size, 28, Max_node_num * (36*2)
        messages = torch.concat((messages, feature_self), dim = -1)
        #---------------------------------------------------------------
        #INPUT DIMEN:Batch-size, 28, Max_node_num, 72
        messages = self.linear_hidden(messages)
        if self.drop_out:
            messages = F.dropout(messages, training=self.training, p=0.2)
        #SHAPE: Batch-size, 28, Max_node_num, hidden_dimen
        messages = self.acti_func(messages) 
        
        #SHAPE: Batch-size, 28, Max_node_num, Output_dimen
        output = self.out_layer(messages)
        if self.drop_out:
            output = F.dropout(output, training=self.training, p =0.2)
        output = self.acti_func(output) 
        
        output_structure = self.linear_structure(messages)
        if self.drop_out:
            output_structure = F.dropout(output_structure, training=self.training, p=0.2)
        output_structure = self.acti_func(output_structure).unsqueeze(-1)
        
        
        return output, output_structure

In [10]:
class PGNN(nn.Module):
    def __init__(self, hidden_dimen, output_dimen, anchor_num, layer_num = 1, drop_out = True):
        super(PGNN, self).__init__()
        self.drop_out = drop_out
        self.layer_num = layer_num
        self.anchor_num = anchor_num
        if self.layer_num == 1:
            self.pgnn_operate_1 = PGNN_Layer(hidden_dimen, output_dimen, anchor_num, drop_out)
        
            
        if self.layer_num > 1:
            self.pgnn_operate_1 = nn.ModuleList([PGNN_Layer(hidden_dimen, output_dimen, anchor_num, drop_out) for i in range(0, layer_num)])
           
                
        
    def forward(self, x, dist_max, dist_argmax, mask_dist_max):

        if self.layer_num == 1:
            x_position, x = self.pgnn_operate_1(x, dist_max, dist_argmax, mask_dist_max)
    
            return x_position

        
        if self.layer_num > 1:
            for i in range(self.layer_num):
                _, x = self.pgnn_operate_1[i](x, dist_max, dist_argmax, mask_dist_max)
            x_position = _   

            return x_position

In [11]:
#PGNN处理后数据形状 : Batch_num, 28, Max_node_num, output_dimen ------> (Batch_num * Max_node_num), output_dimen, 28
class CNN_1D(nn.Module):
    def __init__(self, output_dimen, hidden_channels_1, hidden_channels_2):
        super(CNN_1D, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv1d(in_channels = output_dimen, out_channels = hidden_channels_1, kernel_size=7,  stride=1, padding=0),
            nn.LeakyReLU(), #len 22 days
            nn.Conv1d(in_channels = hidden_channels_1, out_channels = hidden_channels_2, kernel_size=7,  stride=1, padding=0),
            nn.LeakyReLU(), #len 16 days
            nn.Conv1d(in_channels = hidden_channels_2, out_channels = hidden_channels_2, kernel_size = 6, stride = 2, padding=1),
            nn.LeakyReLU())  #len: 7 days

        
        for m in self.modules():
            if isinstance(m, nn.Conv1d) or isinstance(m, nn.ConvTranspose1d):
                # 对卷积层使用 Kaiming 正态初始化
                nn.init.kaiming_uniform_(m.weight.data, nonlinearity='leaky_relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
        
       
        
#PGNN处理后数据形状 : Batch_num, 28, Max_node_num, output_dimen ------> (Batch_num * Max_node_num), output_dimen, 28
    
    def forward(self, x):
        batch_size, seq_len, max_node_num, out_dim = x.size()
        x= x.permute(0, 2, 3, 1)
        x = x.flatten(start_dim=0, end_dim=1)
        #output shape: (Batch_num * Max_node_num), hidden_channels_2, 7
        out = self.conv_layer(x) 
        #----------------------------------------------------
        #output shape: (Batch_num * Max_node_num), 7, hidden_channels_2
        out = out.permute(0, 2, 1)
        #------------------------------------------------------
       
        return out

In [12]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, output_seq_len, num_nodes, drop_out = True):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers = 1, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size * output_seq_len)
        self.num_nodes = num_nodes
        self.output_seq_len = output_seq_len
        self.output_size = output_size
        self.acti_func = nn.ReLU()
        self.drop_out = drop_out
        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.weight.data = init.kaiming_uniform_(m.weight.data, nonlinearity='relu')
                if m.bias is not None:
                    m.bias.data = init.constant_(m.bias.data, 0.0) 
          
    
    def forward(self, x):
        # x shape: (Batch_num * Max_node_num), 7, hidden_size
        lstm_out, _ = self.lstm(x)
        if self.drop_out:
            lstm_out = F.dropout(lstm_out, training=self.training, p=0.2)
        lstm_out = self.acti_func(lstm_out)
        # lstm_out shape: (Batch_num * Max_node_num), hidden_size 
        lstm_out = lstm_out[:, -1, :]
        out = self.fc(lstm_out)  
        out= self.acti_func(out)
        #out shape: Batch_num, Max_node_num, output_seq_len
        out = out.view(-1, self.num_nodes, self.output_seq_len).permute(0, 2, 1)
        
        return out

In [65]:
#----------------------------------------------------------------------
#PGNN
anchor_num = 50
num_nodes = 1466
hidden_dimen = 128
output_dimen = 64
#-------------------------------------------------
#CNN
hidden_channels_1, hidden_channels_2 = 128, 64
#-------------------------------------------------
input_size = hidden_channels_2  # LSTM 输入的维度
hidden_size = 128   # LSTM 隐藏层的维度
output_seq_len = 14  # 预测14天
output_size = 1

#-------------------------------------------------
#Domain Classifier
class_hidden_1, class_hidden_2 = 258, 128
k = 500
alpha = 0.35
beta= 0.30

In [70]:
#----------------------------------------------------------------------------------------------------
classifier = Domain_Classifier(hidden_channels_2, class_hidden_1, class_hidden_2, k, num_nodes, alpha)
classifier_optimizer = optim.NAdam(classifier.parameters(), lr=3e-4, weight_decay=1e-5)
classifier_scheduler = optim.lr_scheduler.StepLR(classifier_optimizer, step_size = 11, gamma = 0.55)
criterion_BCE = nn.BCEWithLogitsLoss()

In [71]:
lstm = LSTMModel(input_size, hidden_size, output_size, output_seq_len, num_nodes)
lstm_optimizer = optim.NAdam(lstm.parameters(), lr= 2e-4, weight_decay=1e-5) #Nadam 4e-4
lstm_scheduler = optim.lr_scheduler.StepLR(lstm_optimizer, step_size=11, gamma=0.6)
#------------------------------------------------------------------------------------------
pgnn = PGNN(hidden_dimen, output_dimen, anchor_num)
cnn_pgnn_optimizer = optim.NAdam(list(pgnn.parameters()), lr= 2e-4, weight_decay=1e-5) #Nadam 6e-4
cnn_pgnn_scheduler = optim.lr_scheduler.StepLR(cnn_pgnn_optimizer, step_size=11, gamma=0.55)
#------------------------------------------------------------------------------------------
cnn = CNN_1D(output_dimen, hidden_channels_1, hidden_channels_2)
cnn_optimizer = optim.NAdam(list(cnn.parameters()), lr= 2e-4, weight_decay=1e-5) #Nadam 6e-4
cnn_scheduler = optim.lr_scheduler.StepLR(cnn_optimizer, step_size=11, gamma=0.55)
#----------------------------------------------------------------------------------------------------
criterion_MSE = nn.MSELoss(reduction='none')  # 使用均方误差作为损失函数
criterion_MAE = nn.L1Loss(reduction='none')

In [72]:
checkpoint = torch.load('D:/ThesisData/processed data/ModelPara/source_pgnn_cnn_lstm_14days.pth')
pgnn.load_state_dict(checkpoint['pgnn_state_dict'])
cnn.load_state_dict(checkpoint['cnn_state_dict'])
lstm.load_state_dict(checkpoint['rnn_state_dict'])
#cnn_pgnn_optimizer.load_state_dict(checkpoint['cnn_pgnn_optimizer_state_dict'])
#cnn_optimizer.load_state_dict(checkpoint['cnn_optimizer_state_dict'])
#lstm_optimizer.load_state_dict(checkpoint['lstm_optimizer_state_dict'])

<All keys matched successfully>

In [47]:
source_train = torch.load("D:/ThesisData/processed data/SourceDomain/NEW/train_data_14days.h5")
target_train = torch.load("D:/ThesisData/processed data/TargetDomain/NEW/train_data_14days.h5")
vali_barcelona = torch.load("D:/ThesisData/processed data/TargetDomain/NEW/Barcelona_vali_data_14days.h5")
test_barcelona = torch.load("D:/ThesisData/processed data/TargetDomain/NEW/Barcelona_test_data_14days.h5")

In [56]:
indices = list(range(len(source_train) // 4))  # 前半部分数据的索引
subset_source = Subset(source_train, indices) 
combined_train = ConcatDataset([subset_source, target_train])

In [73]:
batch_size = 14
train_loader = DataLoader(combined_train, batch_size=batch_size, shuffle=True)
vali_loader = DataLoader(vali_barcelona, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_barcelona, batch_size=batch_size, shuffle=False)

In [74]:
epoch_num = 40

In [75]:
for h in range(epoch_num):


    lstm.train()
    pgnn.train()
    cnn.train()
    classifier.train()  

    
    if (h + 1) % 4 != 0:
        for batch_input, batch_labels, batch_dist, batch_dist_arg, batch_masks, batch_min, batch_max, batch_dist_mask, batch_class_labels in train_loader:

        
            batch_input, batch_labels, batch_masks = batch_input.squeeze(), batch_labels.squeeze(), batch_masks.squeeze()
            batch_min, batch_max = batch_min.squeeze(), batch_max.squeeze()
  
        
            pgnn_output = pgnn(batch_input, batch_dist, batch_dist_arg, batch_dist_mask)
            cnn_output = cnn(pgnn_output)
            #---------------------------------------------------------------------
            #class_input = GradientReversalLayer.apply(cnn_output, alpha)
            batch_class = classifier(cnn_output)
            #---------------------------------------------------------------------
            loss_bce = criterion_BCE(batch_class, batch_class_labels)
            #---------------------------------------------------------------------
        

            #---------------------------------------------------------------------
            classifier_optimizer.zero_grad()
            loss_bce.backward()

            torch.nn.utils.clip_grad_norm_(classifier.parameters(), 1.0)
            

        
        print("train domain classifier:")
        print(f"epoch {h + 1}, Train Domain Classifier: BCE={loss_bce.item():.3f}")  
        
    
    else:
        for batch_input, batch_labels, batch_dist, batch_dist_arg, batch_masks, batch_min, batch_max, batch_dist_mask, batch_class_labels in train_loader:

            
            batch_input, batch_labels, batch_masks = batch_input.squeeze(), batch_labels.squeeze(), batch_masks.squeeze()
            batch_min, batch_max = batch_min.squeeze(), batch_max.squeeze()


            pgnn_output = pgnn(batch_input, batch_dist, batch_dist_arg, batch_dist_mask)
            cnn_output = cnn(pgnn_output)
            batch_outputs = lstm(cnn_output)  
            #---------------------------------------------------------------------
            batch_class = classifier(cnn_output)
            #---------------------------------------------------------------------
            loss_bce = criterion_BCE(batch_class, batch_class_labels)
            #---------------------------------------------------------------------
            loss_mse = criterion_MSE(batch_outputs, batch_labels) * batch_masks
            loss_mae = criterion_MAE(batch_outputs, batch_labels) * batch_masks
            loss_mse = loss_mse.sum() / batch_masks.sum()
            loss_mae = loss_mae.sum() / batch_masks.sum()      
            #----------------------------------------------------------------------
        

            #----------------------------------------------------------------------
            lstm_optimizer.zero_grad()
            cnn_optimizer.zero_grad()
            cnn_pgnn_optimizer.zero_grad()
            #----------------------------------------------------------------------
            
    
            #----------------------------------------------------------------------
            loss_total = loss_mse + beta * loss_bce
            loss_total.backward()

            
            torch.nn.utils.clip_grad_norm_(classifier.parameters(), 1.0)
            torch.nn.utils.clip_grad_norm_(lstm.parameters(), 1.0)
            torch.nn.utils.clip_grad_norm_(cnn.parameters(), 1.0)
            torch.nn.utils.clip_grad_norm_(pgnn.parameters(), 1.0)

            

        
        #----------------------------------------------------------------------
        inverse_outputs = batch_outputs * (batch_max - batch_min) + batch_min
        inverse_labels = batch_labels * (batch_max - batch_min) + batch_min

        
        loss_mse = criterion_MSE(inverse_outputs, inverse_labels) * batch_masks
        loss_mae = criterion_MAE(inverse_outputs, inverse_labels) * batch_masks
        
        
        
        loss_mse = loss_mse.sum() / batch_masks.sum()
        loss_mae = loss_mae.sum() / batch_masks.sum()    
        
        del batch_outputs
        del batch_labels
        del inverse_outputs
        del inverse_labels
        del batch_min
        del batch_max

        if (h + 1) % 4 == 0: 
            print("--------------------------------------------")
            print("train feature extractor:")
            print(f"epoch {h + 1}, Train Barcelona: MSE={loss_mse.item():.1f}, MAE={loss_mae.item():.1f}")            
       

        
#--------------------------------------------------
    lstm.eval()
    pgnn.eval()
    cnn.eval()  
    classifier.eval()

    
    with torch.no_grad():
        if (h + 1) % 4 == 0: 
            for batch_input, batch_labels, batch_dist, batch_dist_arg, batch_masks, batch_min, batch_max, batch_dist_mask, _ in vali_loader:

                batch_input, batch_labels, batch_masks = batch_input.squeeze(), batch_labels.squeeze(), batch_masks.squeeze()
                batch_min, batch_max = batch_min.squeeze(), batch_max.squeeze()

                pgnn_output = pgnn(batch_input, batch_dist, batch_dist_arg, batch_dist_mask)
                cnn_output = cnn(pgnn_output)
        
                batch_outputs = lstm(cnn_output)
          
    
                #----------------------------------------------------------------------
                Barcelona_vali_outputs = batch_outputs * (batch_max - batch_min) + batch_min
                Barcelona_vali_labels = batch_labels * (batch_max - batch_min) + batch_min

                loss_mse = criterion_MSE(Barcelona_vali_outputs, Barcelona_vali_labels) * batch_masks
                loss_mae = criterion_MAE(Barcelona_vali_outputs, Barcelona_vali_labels) * batch_masks
                loss_mse_ = loss_mse.sum() / batch_masks.sum()
                loss_mae = loss_mae.sum() / batch_masks.sum()    

            
            print(f"----Validation Barcelona: MSE={loss_mse_.item():.1f}, MAE={loss_mae.item():.1f}")
         

            for batch_input, batch_labels, batch_dist, batch_dist_arg, batch_masks, batch_min, batch_max, batch_dist_mask, _ in test_loader:

                batch_input, batch_labels, batch_masks = batch_input.squeeze(), batch_labels.squeeze(), batch_masks.squeeze()
                batch_min, batch_max = batch_min.squeeze(), batch_max.squeeze()

                pgnn_output = pgnn(batch_input, batch_dist, batch_dist_arg, batch_dist_mask)
                cnn_output = cnn(pgnn_output)
        
                batch_outputs = lstm(cnn_output)
          
    
                #----------------------------------------------------------------------
                Barcelona_test_outputs = batch_outputs * (batch_max - batch_min) + batch_min
                Barcelona_test_labels = batch_labels * (batch_max - batch_min) + batch_min

                loss_mse = criterion_MSE(Barcelona_test_outputs, Barcelona_test_labels) * batch_masks
                loss_mae = criterion_MAE(Barcelona_test_outputs, Barcelona_test_labels) * batch_masks
                loss_mse = loss_mse.sum() / batch_masks.sum()
                loss_mae = loss_mae.sum() / batch_masks.sum()    

            
            print(f"----Test Barcelona: MSE={loss_mse.item():.1f}, MAE={loss_mae.item():.1f}")
            print("--------------------------------------------")



    

            if h == epoch_num-1 or loss_mse_ <=1000:
            #if h == epoch_num-1 or (loss_mse_ <=1293 and h > 0):
                pgnn_cnn_results = {"Barcelona_vali": Barcelona_vali_outputs.detach().numpy(), "Barcelona_vali_label": Barcelona_vali_labels.detach().numpy(),
                                    "Barcelona_test": Barcelona_test_outputs.detach().numpy(), "Barcelona_test_label": Barcelona_test_labels.detach().numpy(),
                                    "Barcelona_node_num": 1273}

                save_data(pgnn_cnn_results,"D:/ThesisData/processed data/TargetDomain/NEW/results/pgnn_cnn_lstm_gan_results.h5" )    
                break

    classifier_optimizer.step()
    lstm_optimizer.step()
    cnn_optimizer.step()
    cnn_pgnn_optimizer.step()    


train domain classifier:
epoch 1, Train Domain Classifier: BCE=0.694
train domain classifier:
epoch 2, Train Domain Classifier: BCE=8.843
train domain classifier:
epoch 3, Train Domain Classifier: BCE=0.693
--------------------------------------------
train feature extractor:
epoch 4, Train Barcelona: MSE=4571.6, MAE=48.4
----Validation Barcelona: MSE=1063.5, MAE=19.9
----Test Barcelona: MSE=1647.0, MAE=26.3
--------------------------------------------
train domain classifier:
epoch 5, Train Domain Classifier: BCE=0.685
train domain classifier:
epoch 6, Train Domain Classifier: BCE=0.657
train domain classifier:
epoch 7, Train Domain Classifier: BCE=0.742
--------------------------------------------
train feature extractor:
epoch 8, Train Barcelona: MSE=4631.2, MAE=50.0
----Validation Barcelona: MSE=1005.3, MAE=19.8
----Test Barcelona: MSE=1507.0, MAE=25.5
--------------------------------------------
train domain classifier:
epoch 9, Train Domain Classifier: BCE=23.020
train domain cla

In [67]:
torch.save({
    'pgnn_state_dict': pgnn.state_dict(),
    'lstm_state_dict': lstm.state_dict(),
    'cnn_state_dict': cnn.state_dict(),
    'classifier_state_dict': classifier.state_dict(),
    'lstm_optimizer_state_dict': lstm_optimizer.state_dict(),
    'cnn_pgnn_optimizer_state_dict': cnn_pgnn_optimizer.state_dict(),
    'classifier_optimizer_state_dict': classifier_optimizer.state_dict(),
}, 'D:/ThesisData/processed data/ModelPara/source_pgnn_cnn_lstm_gan.pth')